In [1]:
import warnings as wns
wns.filterwarnings('ignore')

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


단어 품사 구분하기  
'I work at google', 'I google at work' 의 단어 품사를 구분하눈 RNN 코드응 텐서플로우로 구현한다.  
각 단어는 one hot encoding으로 표현한다.

In [3]:
# I      => [1, 0, 0, 0] 대명사
# work   => [0, 1, 0, 0] 동사
# at     => [0, 0, 1, 0] 전치사
# google => [0, 0, 0, 1] 명사

# I work at google => [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
# I google at work => [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]]

inputs = np.array([
    [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]],
    [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]]
])
inputs

array([[[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]],

       [[1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 0, 0]]])

In [8]:
tf.reset_default_graph()
tf.set_random_seed(21)
tf_inputs = tf.constant(inputs, dtype=tf.float32)
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=3)
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype=tf.float32, inputs=tf_inputs)

print('출력값 개수: {}'.format(outputs))
print('상태값 개수: {}'.format(state))
print('=' * 80)

print('가중치 개수와 편향값 개수')
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)
print('=' * 80)

variable_names = [v.name for v in tf.trainable_variables()]
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])
    print('출력값\n{}\n'.format(output_run))
    print('상태값\n{}\n'.format(state_run))
    print('가중치와 편향값')
    values = sess.run(variable_names)
    for k, v in zip(variable_names, values):
        print('{}\n{}'.format(k, v))

출력값 개수: Tensor("rnn/transpose_1:0", shape=(2, 4, 3), dtype=float32)
상태값 개수: Tensor("rnn/while/Exit_3:0", shape=(2, 3), dtype=float32)
가중치 개수와 편향값 개수
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>
출력값
[[[ 0.11188304  0.48046544  0.63265073]
  [-0.69595605  0.08518933  0.8141431 ]
  [ 0.61908203 -0.32562548  0.933208  ]
  [ 0.12918115  0.89260894  0.7059085 ]]

 [[ 0.11188304  0.48046544  0.63265073]
  [-0.12850831  0.60322356  0.8797253 ]
  [ 0.08233774 -0.00408355  0.93313795]
  [-0.49893805  0.3457571   0.8076218 ]]]

상태값
[[ 0.12918115  0.89260894  0.7059085 ]
 [-0.49893805  0.3457571   0.8076218 ]]

가중치와 편향값
rnn/basic_rnn_cell/kernel:0
[[ 0.11235344  0.52358925  0.7458236 ]
 [-0.41523334 -0.2798797   0.48265624]
 [ 0.5138159  -0.32121253  0.6416223 ]
 [ 0.31495857  0.3329233   0.71801186]
 [-0.501872    0.7233957  -0.57018113]
 [-0.67484295 -0.22664434  0.5303705 ]
 [-0.10083246  0.6215

첫 단어 이전에는 이전 상태값이 존재하지 않기 때문에 두 문장의 첫 단어 출력값이 동일하다.  
[ 0.11235344  0.52358925  0.7458236 ]  
  
이전 상태값이 현재 출력값에 영향을 주기 때문에 두 번째 단어부터의 출력값은 두 문장이 다르게 나타난다.  
[-0.69595605  0.08518933  0.8141431 ]  
[ 0.61908203 -0.32562548  0.933208  ]  
[ 0.12918115  0.89260894  0.7059085 ] # 마지막 단어 출력값  
  
[-0.12850831  0.60322356  0.8797253 ]  
[ 0.08233774 -0.00408355  0.93313795]  
[-0.49893805  0.3457571   0.8076218 ] # 마지막 단어 출력값  
  
상태값은 항상 마지막 단어의 출력값과 동일하다.  
[ 0.12918115  0.89260894  0.7059085 ]  
[-0.49893805  0.3457571   0.8076218 ] 